In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 22 15:56:47 2021

@author: Anirban
"""


import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
#from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime


model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

model.save('mymodel.h5',model_saved)

#To test for individual images




Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


C:\Users\ANIRBA~1\AppData\Local\Temp/ipykernel_105876/2913991373.py:58: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10
83/83 [==============================] - 45s 536ms/step - loss: 0.4023 - accuracy: 0.8120 - val_loss: 0.1202 - val_accuracy: 0.9485
Epoch 2/10
83/83 [==============================] - 22s 269ms/step - loss: 0.1572 - accuracy: 0.9444 - val_loss: 0.1389 - val_accuracy: 0.9588
Epoch 3/10
83/83 [==============================] - 23s 276ms/step - loss: 0.1405 - accuracy: 0.9589 - val_loss: 0.1080 - val_accuracy: 0.9536
Epoch 4/10
83/83 [==============================] - 24s 293ms/step - loss: 0.1249 - accuracy: 0.9543 - val_loss: 0.1249 - val_accuracy: 0.9330
Epoch 5/10
83/83 [==============================] - 23s 278ms/step - loss: 0.0919 - accuracy: 0.9673 - val_loss: 0.1261 - val_accuracy: 0.9588
Epoch 6/10
83/83 [==============================] - 22s 265ms/step - loss: 0.0789 - accuracy: 0.9749 - val_loss: 0.1065 - val_accuracy: 0.9588
Epoch 7/10
83/83 [==============================] - 23s 275ms/step - loss: 0.0746 - accuracy: 0.9734 - val_loss: 0.0644 - val_accuracy: 0.9691

In [4]:
mymodel=load_model('mymodel.h5')
test_image=image.load_img(r'test/with_mask/1-with-mask.jpg',
                          target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]


# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()